### Relevant links

https://www.kaggle.com/c/cat-in-the-dat-ii/

### Imports

In [19]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline 
%load_ext autotime

time: 685 µs (started: 2021-01-23 08:31:16 +00:00)


In [63]:
import pandas as pd
import sys
import category_encoders as ce
from sklearn.pipeline import Pipeline
sys.path.insert(1,'/home/jupyter/Learning/kaggle/cat-in-the-dat-ii/src') # to import utils
from utils import advanced_describe # from src

time: 934 µs (started: 2021-01-23 08:52:46 +00:00)


In [64]:
pd.options.display.max_rows = 4000
pd.set_option('display.max_columns', 999)
pd.options.mode.chained_assignment = None

time: 943 µs (started: 2021-01-23 08:52:47 +00:00)


### Getting data

In [65]:
train = pd.read_csv('../inputs/train.csv')
test = pd.read_csv('../inputs/test.csv')
ss = pd.read_csv('../inputs/sample_submission.csv')

time: 2.92 s (started: 2021-01-23 08:52:48 +00:00)


In [66]:
advanced_describe(train)

,count,mean,std,min,25%,50%,75%,max,missing,missing%,unique,unique_values,dtype
bin_0,582106,0.092301,0.289451,0.0,0.00,0.0,0.00,1.0,17894,0.029823,3,"[0.0, 1.0, nan]",float64
bin_1,581997,0.185532,0.388729,0.0,0.00,0.0,0.00,1.0,18003,0.030005,3,"[0.0, 1.0, nan]",float64
bin_2,582070,0.278704,0.448362,0.0,0.00,0.0,1.00,1.0,17930,0.029883,3,"[0.0, 1.0, nan]",float64
bin_3,581986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18014,0.030023,3,"[F, T, nan]",object
bin_4,581953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18047,0.030078,3,"[N, Y, nan]",object
day,582048,4.112767,2.034430,1.0,2.00,5.0,6.00,7.0,17952,0.029920,8,"[6.0, 7.0, 5.0, 3.0, 1.0, 2.0, nan, 4.0]",float64
id,600000,299999.500000,173205.225094,0.0,149999.75,299999.5,449999.25,599999.0,0,0.000000,600000,NaN,int64
month,582012,6.371317,3.458959,1.0,3.00,6.0,8.00,12.0,17988,0.029980,13,"[3.0, 7.0, 9.0, 12.0, 4.0, 6.0, 1.0, 8.0, 5.0,...",float64
nom_0,581748,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18252,0.030420,4,"[Red, Blue, Green, nan]",object
nom_1,581844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18156,0.030260,7,"[Trapezoid, Star, nan, Circle, Triangle, Polyg...",object


time: 4.06 s (started: 2021-01-23 08:52:51 +00:00)


### Missing value treatment

In [67]:
cat_features = train.drop(["id", "target"], axis=1).select_dtypes(exclude = 'number').columns.tolist()
cont_features = train.drop(["id", "target"], axis=1).select_dtypes(include = 'number').columns.tolist()

time: 221 ms (started: 2021-01-23 08:52:55 +00:00)


In [68]:
for col in cat_features:
    train[col].fillna('missing', inplace=True)
    test[col].fillna('missing', inplace=True)
for col in cont_features:
    train[col].fillna(-99999, inplace=True)
    test[col].fillna(-99999, inplace=True)
advanced_describe(train)

,count,mean,std,min,25%,50%,75%,max,missing,missing%,unique,unique_values,dtype
bin_0,600000,-2982.213962,17009.828068,-99999.0,0.00,0.0,0.00,1.0,0,0.0,3,"[0.0, 1.0, -99999.0]",float64
bin_1,600000,-3000.290030,17059.974913,-99999.0,0.00,0.0,0.00,1.0,0,0.0,3,"[0.0, 1.0, -99999.0]",float64
bin_2,600000,-2988.033075,17026.435284,-99999.0,0.00,0.0,1.00,1.0,0,0.0,3,"[0.0, 1.0, -99999.0]",float64
bin_3,600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,3,"[F, T, missing]",object
bin_4,600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,3,"[N, Y, missing]",object
day,600000,-2987.980367,17037.209089,-99999.0,2.00,4.0,6.00,7.0,0,0.0,8,"[6.0, 7.0, 5.0, 3.0, 1.0, 2.0, -99999.0, 4.0]",float64
id,600000,299999.500000,173205.225094,0.0,149999.75,299999.5,449999.25,599999.0,0,0.0,600000,NaN,int64
month,600000,-2991.789715,17054.141258,-99999.0,3.00,6.0,8.00,12.0,0,0.0,13,"[3.0, 7.0, 9.0, 12.0, 4.0, 6.0, 1.0, 8.0, 5.0,...",float64
nom_0,600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,4,"[Red, Blue, Green, missing]",object
nom_1,600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,7,"[Trapezoid, Star, missing, Circle, Triangle, P...",object


time: 4.95 s (started: 2021-01-23 08:52:55 +00:00)


### Cat encoding

In [69]:
# Target encoding requires to be done separately for each cross validation step
# Should nicey fit in the train.py job since each fold in called separately in model_training.sh

time: 563 µs (started: 2021-01-23 08:53:05 +00:00)


In [70]:
# Construct unique value dict
uv_dict = dict()
for col in cat_features:
    uv_dict[col] = train[col].nunique()
uv_dict

{'bin_3': 3,
 'bin_4': 3,
 'nom_0': 4,
 'nom_1': 7,
 'nom_2': 7,
 'nom_3': 7,
 'nom_4': 5,
 'nom_5': 1221,
 'nom_6': 1520,
 'nom_7': 223,
 'nom_8': 223,
 'nom_9': 2219,
 'ord_1': 6,
 'ord_2': 7,
 'ord_3': 16,
 'ord_4': 27,
 'ord_5': 191}

time: 1.09 s (started: 2021-01-23 08:53:09 +00:00)


In [71]:
# Encode
te_list = []
ohe_list = []

for col in cat_features:
    if uv_dict[col] >=6:
        # run target encoding
        te_list.append(col)
    else:
        # run one hot encoding
        ohe_list.append(col)
        
print("Target encoding columns: ",te_list)
print("One hot encoding columns: ",ohe_list)

Target encoding columns:  ['nom_1', 'nom_2', 'nom_3', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']
One hot encoding columns:  ['bin_3', 'bin_4', 'nom_0', 'nom_4']
time: 1.31 ms (started: 2021-01-23 08:53:17 +00:00)


In [72]:
# See new values in test data
train_val = dict()
test_val = dict()
diff_val = dict()
for col in cat_features:
    train_val[col] = set(train[col].unique())
    test_val[col] = set(test[col].unique())
    diff_val[col] =  test_val[col].difference(train_val[col])
diff_val

{'bin_3': set(),
 'bin_4': set(),
 'nom_0': set(),
 'nom_1': set(),
 'nom_2': set(),
 'nom_3': set(),
 'nom_4': set(),
 'nom_5': set(),
 'nom_6': {'a885aacec'},
 'nom_7': set(),
 'nom_8': set(),
 'nom_9': set(),
 'ord_1': set(),
 'ord_2': set(),
 'ord_3': set(),
 'ord_4': set(),
 'ord_5': set()}

time: 954 ms (started: 2021-01-23 08:53:21 +00:00)


In [82]:
# run target encoding and one hot encoding using pipeline (for simplicity and avoiding temp table creations)
encoding_pipeline = Pipeline([
  ('ohe', ce.OneHotEncoder(cols=ohe_list, use_cat_names=True, handle_unknown = 'error', return_df=True)),
  ('te', ce.TargetEncoder(cols=te_list, smoothing=0, handle_unknown = 'value', return_df=True))
])

# target encoding - from comments in source code
# For the case of categorical target: features are replaced with a blend of posterior probability of the target
# given particular categorical value and the prior probability of the target over all the training data.

# Get the encoded training set:
train_encoded = encoding_pipeline.fit_transform(train, train['target'])

# Get the encoded test set, not no target passed!
test['target'] = 0
test_encoded = encoding_pipeline.transform(test)

/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/opt/conda/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


time: 10.3 s (started: 2021-01-23 09:00:26 +00:00)


In [83]:
train_encoded.head(3)

,id,bin_0,bin_1,bin_2,bin_3_F,bin_3_T,bin_3_missing,bin_4_N,bin_4_Y,bin_4_missing,nom_0_Red,nom_0_Blue,nom_0_Green,nom_0_missing,nom_1,nom_2,nom_3,nom_4_Bassoon,nom_4_Theremin,nom_4_missing,nom_4_Oboe,nom_4_Piano,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,1,0,0,1,0,0,1,0,0,0,0.225464,0.168014,0.219534,1,0,0,0,0,0.209756,0.161017,0.103301,0.128509,0.164251,3.0,0.172972,0.205176,0.113524,0.222938,0.143608,6.0,3.0,0
1,1,1.0,1.0,0.0,1,0,0,0,1,0,1,0,0,0,0.130201,0.201104,0.185862,0,1,0,0,0,0.193767,0.200913,0.129012,0.197291,0.152830,3.0,0.232669,0.175404,0.135513,0.233747,0.236897,7.0,7.0,0
2,2,0.0,1.0,0.0,1,0,0,1,0,0,1,0,0,0,0.182805,0.168014,0.159849,1,0,0,0,0,0.181311,0.163690,0.262747,0.172607,0.187849,3.0,0.187406,0.132253,0.289383,0.201003,0.208738,5.0,9.0,0


time: 22.9 ms (started: 2021-01-23 09:00:37 +00:00)


In [84]:
test_encoded[test.nom_6 == 'a885aacec'].head(3)

,id,bin_0,bin_1,bin_2,bin_3_F,bin_3_T,bin_3_missing,bin_4_N,bin_4_Y,bin_4_missing,nom_0_Red,nom_0_Blue,nom_0_Green,nom_0_missing,nom_1,nom_2,nom_3,nom_4_Bassoon,nom_4_Theremin,nom_4_missing,nom_4_Oboe,nom_4_Piano,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
343202,943202,0.0,0.0,0.0,1,0,0,0,0,1,1,0,0,0,0.15736,0.168014,0.178947,0,1,0,0,0,0.161961,0.187205,0.186924,0.203241,0.208914,3.0,0.172972,0.152205,0.182082,0.242218,0.163265,-99999.0,1.0,0


time: 69.1 ms (started: 2021-01-23 09:00:37 +00:00)
